In [22]:
import numpy as np
from functools import reduce

from validation import prob_of_error_propagation
from src.scrapper import parse_conllu_file
from src.tagger import HiddenMarkovModel, HiddenMarkovModelTagger
from src.visualization import plot_viterbi_path_binary, plot_viterbi_matrix

In [23]:
train = parse_conllu_file(filepath="../datasets/en_partut-ud-train.conllu")
test = parse_conllu_file(filepath="../datasets/en_partut-ud-test.conllu")
tagger = HiddenMarkovModel(corpus=train).train()

In [24]:
corpus = test
corpus_p = []
for sentence in test:
    s = reduce(lambda x, y: x + ' ' + y, map(lambda x: x[0], sentence))
    _, s_p, _ = tagger.viterbi_best_path(s)
    corpus_p.append(s_p)

print(prob_of_error_propagation(corpus, corpus_p))

0.2673684210526316


In [30]:
def predict_all(corpus, tagger):
    corpus_p = []
    for sentence in corpus:
        s = reduce(lambda x, y: x + ' ' + y, map(lambda x: x[0], sentence))
        _, s_p, _ = tagger.viterbi_best_path(s)
        corpus_p.append(s_p)

    return corpus_p

In [40]:
def get_confusion_matrix(corpus, corpus_p, tagset):
    N = len(tagset)
    cm = np.zeros((N, N))
    for i in range(len(corpus)):
        expected, prediction = corpus[i], corpus_p[i]
        for token, token_p in zip(map(lambda x: x[1], expected), map(lambda x: x[1], prediction)):
            cm[tagset.index(token), tagset.index(token_p)] += 1
    
    return cm

In [61]:
cm = get_confusion_matrix(corpus, corpus_p, tagger.tagset)
cm_formatted = np.array2string(cm, precision=0, separator=' ', suppress_small=True, max_line_width=100)
print(cm_formatted)

[[  1.   0.   0.   0.   0.   0.   0.   0.   1.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  1. 732.   0.   0.   9.   0.   0.   9.   2.   0.   0.   0.   0.   0.   0.   0.   0.   1.]
 [  0.   0.  29.   0.   0.   0.   7.   0.   0.   0.   0.   0.   0.   0.   0.   2.  13.   0.]
 [  6.   0.   0. 333.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  1.  74.   0.   0. 131.   0.   1.  13.   1.   0.   0.   0.   0.   0.   0.   1.   1.   0.]
 [  0.   2.   0.   0.   0.   1.   0.   1.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  1.   1.   1.   0.   2.   0.  86.   0.   0.   0.   0.   2.  16.   0.   0.   0.   0.   0.]
 [  2.  95.   0.   0.   3.   0.   0. 212.   0.   0.   0.   0.   0.   0.   0.   0.   1.  13.]
 [  0.  45.   0.   0.   6.   0.   0.   0.  39.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  1.   0.   0.   0.   0.   0.   0.   0.   0.  94.   0.   0.   0.   0.   0.   0.   1.   0.]
 [  0.   1.   0.   0.   0.   0.   0.   1.   0.   0.   0.   0.   0.   0

In [62]:
def get_accuracy(confusion_matrix):
    # get the matrix diagonal and sum all the correct predictions (true positives for each tag) 
    diagonal = np.diagonal(confusion_matrix)
    total_correct = np.sum(diagonal)
    
    # sum the total number of predictions made
    total_predictions = np.sum(confusion_matrix)
    
    # accuracy is defined as the ratio of correct predictions made
    accuracy = total_correct / total_predictions
    
    return accuracy

In [75]:
def get_precision(confusion_matrix):
    # number of tags
    total_tags = len(confusion_matrix)

    precisions, predictions = [], []

    for i in range(total_tags):
        # each column contains all the info we need
        total_tag_correct = confusion_matrix[i, i]  # tp for that tag
        total_tag_predictions = np.sum(confusion_matrix[:, i])

        # precision is defined as the ratio of true predictions from all positive (applies to a certain tag)
        if total_tag_predictions == 0:  # some tags may not appear in the test set at all 
            precision = 0
        else:
            precision = total_tag_correct / total_tag_predictions
        precisions.append(precision)
        predictions.append(int(total_tag_predictions))

    return precisions, predictions

In [86]:
print("MODEL ACCURACY")
print("--------------")
acc = get_accuracy(cm)
print(f"Model accuracy: {acc:.4f}")
print()

prec, preds = get_precision(cm)
prec_data = zip(tagger.tagset, prec, preds)
print("MODEL PRECISION")
print("---------------")
for tag in prec_data:
    print(f"{tag[0]}: {tag[1]:.4f} over {int(tag[2])} predictions.")

MODEL ACCURACY
--------------
Model accuracy: 0.8602

MODEL PRECISION
---------------
x: 0.0556 over 18 predictions.
noun: 0.7342 over 997 predictions.
sconj: 0.9355 over 31 predictions.
punct: 0.9970 over 334 predictions.
adj: 0.8239 over 159 predictions.
_: 1.0000 over 1 predictions.
pron: 0.8687 over 99 predictions.
verb: 0.8092 over 262 predictions.
propn: 0.9070 over 43 predictions.
cconj: 1.0000 over 94 predictions.
intj: 0.0000 over 0 predictions.
num: 0.9545 over 44 predictions.
det: 0.9602 over 452 predictions.
sym: 0.0000 over 0 predictions.
part: 1.0000 over 31 predictions.
adv: 0.9589 over 73 predictions.
adp: 0.8899 over 545 predictions.
aux: 0.9389 over 229 predictions.
